# 커널 시작 시 먼저 실행
---

```python
# !pip install -q emoji
# !pip install -q konlpy
# !pip install -qU langchain langchain-core langchain-community langchain-openai
# !pip install -qU huggingface_hub langchain_huggingface
```

```python
from FashionTrendCrawling import FashionTrendCrawling as ftc
from KeywordCounter import KeywordCounter as kc
from FashionChatbot import FashionChatbot
from IPython.display import clear_output
import pandas as pd
import time
```

## 부가 기능 함수 정의

### 진행 막대 표시

```python
def progressbar(startTime:float, progress:float, title:str = ""):
    """
    진행 막대 표시
    """
    remainingTime = (time.perf_counter() - startTime) / progress * (1 - progress)
    clear_output(wait = True)
    print(title)
    print(f"{"■" *  round(20 * progress)}{"□" * (20 - round(20 * progress))}", end = " ")
    print(f"{int(remainingTime // 3600):02d}:{int(remainingTime % 3600 // 60):02d}:{int(remainingTime % 60):02d}")
```

### 토큰 비용 출력

```python
def TokenPrice(response_metadata:dict):
    """
    토큰 비용 출력
    """
    if "gpt-4.1-nano" in response_metadata["model_name"]:
        inputToken = 0.2 / 1000000
        outputToken = 0.8 / 1000000
    elif "gpt-4.1-mini" in response_metadata["model_name"]:
        inputToken = 0.8 / 1000000
        outputToken = 3.2 / 1000000
    elif "gpt-4.1" in response_metadata["model_name"]:
        inputToken = 3.0 / 1000000
        outputToken = 12.0 / 1000000
    else:
        print(f"사용 모델 {response_metadata["model_name"]}")
        return

    inputToken = response_metadata["token_usage"]["prompt_tokens"] * inputToken
    outputToken = response_metadata["token_usage"]["completion_tokens"] * outputToken
        
    print("토큰 사용량")
    print(f"입력 토큰 | {response_metadata["token_usage"]["prompt_tokens"]} | US${inputToken}")
    print(f"출력 토큰 | {response_metadata["token_usage"]["completion_tokens"]} | US${outputToken}")
    print(f"총 토큰 | {response_metadata["token_usage"]["total_tokens"]} | US${inputToken + outputToken}")
```

## 정보 수집 함수 정의

### 네이버 블로그 글 수집

```python
def NaverBlogCrawiling(query:str) -> pd.DataFrame:
    """
    네이버 블로그 글 수집

    반환 : pd.DataFrame

    query : 검색 키워드
    """
    print(f"{query} 네이버 블로그 검색")
    
    # 1. a 태그 수집
    elements = ftc.BeginCrawling(
        url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}",
        selectors = [
            "div > div > div > div > div > div.sds-comps-vertical-layout.sds-comps-full-layout.ubuDRz_QzPbskEJRLpc9 > div > div > a"
        ],
        element_names = [
            "블로그 a 태그"
        ],
        timeout = 1000,
        scrolling = True
    )

    # 2. 수집한 a 태그에서 링크 추출
    elements = pd.DataFrame(
        data = ftc.GetSpecific(elements["블로그 a 태그"], "href"),
        columns = ["url"]
    )

    # 3. 추출한 링크에서 iframe 태그 수집
    temp = []
    length = len(elements)
    i = 0
    startTime = time.perf_counter()
    while i < length:
        temp.append(
            ftc.BeginCrawling(
                url = elements.loc[i, "url"],
                selectors = [
                    "iframe"
                ],
                element_names = [
                    "iframe 태그"
                ],
                scrolling = False,
                showLogs = False
            )
        )
        i += 1
        progressbar(startTime, i / length, "추출한 링크에서 iframe 태그 수집")
    temp = ftc.ToDataFrame(temp)

    # 4. 수집한 iframe 태그에서 src 추출
    elements = pd.concat(
        [
            elements,
            pd.Series(ftc.GetSpecific(temp["iframe 태그"], "src"), name = "link")
        ],
        axis = 1
    )

    # 5. 수집한 src를 실제 주소로 변경
    i = 0
    while i < len(elements):
        elements.loc[i, "link"] = "https://m.blog.naver.com/" + elements.loc[i, "link"]
        i += 1

    # 6. 추출한 링크에서 본문 요소 수집
    temp = []
    length = len(elements)
    i = 0
    startTime = time.perf_counter()
    while i < length:
        temp.append(
            ftc.BeginCrawling(
                url = elements.loc[i, "link"],
                selectors = [
                    "#viewTypeSelector > div > div.se-main-container"
                ],
                element_names = [
                    "text"
                ],
                scrolling = False,
                showLogs = False
            )
        )
        i += 1
        progressbar(startTime, i / length, "추출한 링크에서 본문 요소 수집")
    temp = ftc.ToDataFrame(temp)

    # 7. 본문 요소에서 텍스트 추출
    texts = []
    i = 0
    while i < len(temp):
        if pd.isna(temp.loc[i, "text"]):
            texts.append(pd.NA)
        else:
            texts.append(ftc.CleanText(temp.loc[i, "text"].text))
        i += 1
    elements["text"] = texts
    
    # 8. 계측
    length = len(elements)
    i = 0
    while i < length:
        counter.BeginCounting(elements.loc[i, "text"])
        i += 1
        progressbar(startTime, i / length, "키워드 계측")
    
    # 9. 계측 결과 (아이템, 색상, 재질)
    result = counter.GetCounts()

    # 출력물 모두 제거
    clear_output()

    # 반환
    return elements
```

### Vogue 글 수집

```python
def VogueCrawiling() -> pd.DataFrame:
    """
    Vogue 블 수집

    반환 : pd.DataFrame
    """
    print("Vogue 글 수집")
    
    # 1. a 태그 수집
    elements = ftc.BeginCrawling(
        url = "https://www.vogue.co.kr/fashion/fashion-trend/",
        selectors = [
            "div.inner > div > div > ul.d_flex > li > a"
        ],
        element_names = [
            "a 태그"
        ],
        timeout = 1000,
        scrollCountLimit = 5,
        scrolling = True
    )

    # 2. 수집한 a 태그에서 링크 추출
    elements = pd.DataFrame(
        data = ftc.GetSpecific(elements["a 태그"], "href"),
        columns = ["link"]
    )

    # 2. 추출한 href를 실제 링크로 변경
    i = 0
    while i < len(elements):
        elements.loc[i, "link"] = "https://www.vogue.co.kr/" + elements.loc[i, "link"]
        i += 1

    # 3. 추출한 링크에서 본문 요소 수집
    temp = []
    length = len(elements)
    i = 0
    startTime = time.perf_counter()
    while i < length:
        temp.append(
            ftc.BeginCrawling(
                url = elements.loc[i, "link"],
                selectors = [
                    "div.editor > div.contt"
                ],
                element_names = [
                    "text"
                ],
                scrolling = False,
                showLogs = False
            )
        )
        i += 1
        progressbar(startTime, i / length, "추출한 링크에서 본문 요소 수집")
    temp = ftc.ToDataFrame(temp)

    # 4. 본문 요소에서 텍스트 추출
    texts = []
    i = 0
    while i < len(temp):
        if pd.isna(temp.loc[i, "text"]):
            texts.append(pd.NA)
        else:
            texts.append(ftc.CleanText(temp.loc[i, "text"].text))
        i += 1
    elements["text"] = texts
    
    # 5. 계측
    length = len(elements)
    i = 0
    while i < length:
        counter.BeginCounting(elements.loc[i, "text"])
        i += 1
        progressbar(startTime, i / length, "키워드 계측")

    # 출력물 모두 제거
    clear_output()

    # 반환
    return elements
```

### 무신사 이미지, 태그 수집

```python
def MusinsaCrawling() -> tuple[pd.DataFrame, pd.Series]:
    """
    무신사 이미지, 태그 수집

    반환 : (수집한 요소, 태그 빈도수)
    """
    # 1. 해쉬태그, 이미지 수집
    elements = ftc.BeginCrawling(
        url = "https://www.musinsa.com/snap/profile/snap?profile_id=1234726971656693004&snap_id=1268021161197507776",
        selectors = [
            "#commonLayoutContents > main > div > div.SnapDetailSection__Container-sc-44a443ad-0.EZCCk > div > div > div > div > div > div.SnapFeed__ContentContainer-sc-7503dabe-4.eiMZzi > div",
            "#commonLayoutContents > main > div > div.SnapDetailSection__Container-sc-44a443ad-0.EZCCk > div > div > div > div > div > div.gtm-impression-content > div > div > div > div > div > div > img"
        ],
        element_names = [
            "해쉬태그",
            "이미지"
        ],
        timeout = 2000,
        scrollCountLimit = 50,
        scrolling = True
    )

    # 2. 수집한 태그에서 문자열 추출
    temp = []
    i = 0
    while i < len(elements):
        temp.append(elements.loc[i, "해쉬태그"].text.split(" #"))
        # 빈 문자열 제거
        del temp[i][0]
        i += 1
    elements["해쉬태그"] = temp

    # 3. 이미지 태그에서 이미지 추출
    elements["이미지"] = ftc.GetSpecific(elements["이미지"], "src")

    # 4. 태그 수 계측
    counts = dict()
    for tags in elements["해쉬태그"]:
        for tag in tags:
            if tag in counts:
                counts[tag] += 1
            else:
                counts[tag] = 1
    
    # 5. Series로 변환
    counts = pd.Series(counts)

    # 출력물 모두 제거
    clear_output()

    # 반환
    return (elements, counts)
```

# 실행
---

## 시작 단계

```python
# 계측기 생성
counter = kc(
    items = [
        "코트", "패딩", "자켓", "점퍼", "블레이저", "가디건", "니트", "스웨터",
        "셔츠", "블라우스", "티셔츠", "후드티", "바지", "청바지", "슬랙스", "스커트",
        "치마", "원피스", "부츠", "로퍼", "운동화", "스니커즈", "가방", "백팩",
        "머플러", "목도리", "장갑", "모자", "비니",
        "조거팬츠", "바라클라바", "스웻셔츠", "레깅스", "후리스", "롱부츠", "숄더백"
    ],
    colours = [
        "블랙", "화이트", "아이보리", "베이지", "그레이", "네이비", "브라운",
        "카키", "버건디", "레드", "핑크", "블루", "스카이블루", "옐로우", "오렌지",
        "민트", "퍼플",
        "크림", "오트밀", "라떼", "딥그린", "와인", "카멜", "톤온톤"
    ],
    materials = [
        "울", "캐시미어", "가죽", "코튼", "면", "데님", "린넨", "실크", "나일론",
        "폴리에스터", "스웨이드", "니트", "퍼", "벨벳",
        "아크릴", "레이온", "스판덱스", "코듀로이", "플리스",
        "다운", "웰론", "양털", "무스탕"
    ]
)

# 챗봇 참조할 변수
bot = None
```

## 실행 단계

```python
# 사용자 입력
userInput = "겨울 맑은 날 출근 코디"

# 계측기 초기화
counter.ClearCounts()

# 정보 수집
data = NaverBlogCrawiling(userInput.replace(" ", "+"))

# 빈도수 가져오기
counts = counter.GetCounts()

# 챗봇 생성
if bot == None:
    bot = FashionChatbot(max_tokens = 512)
clear_output(wait = True)

# 응답 요청
response = bot.RequestResponse(
f"""
다음 링크의 내용을 참고하라.
{data["link"].str.cat(sep = "\n")}


다음 키워드 언급 횟수를 참고하라.
아이템 언급 수 top5
{"\n".join(f"{index} : {value}" for index, value in counts[0].sort_values(ascending = False).head(5).items())}

색상 언급 수 top5
{"\n".join(f"{index} : {value}" for index, value in counts[1].sort_values(ascending = False).head(5).items())}

재질 언급 수 top5
{"\n".join(f"{index} : {value}" for index, value in counts[2].sort_values(ascending = False).head(5).items())}


블로그 내용을 요약하고 \"{userInput}\"에 대해 추천하라.
"""
)

# 응답 출력
print(f"검색 키워드 >> {userInput}\n{response.content}\n")
TokenPrice(response.response_metadata)

# 추가 질의응답
while True:
    # 구분선
    print("\n", "=" * 50)
    
    # 사용자 입력
    userInput = input("\n추가 질문 >> ")

    # 종료 확인
    if userInput in ["end", "exit", "quit", "stop", "종료", "끝"]:
        break

    # 응답
    response = bot.RequestResponse(userInput)
    print(f"\n응답 >>\n{response.content}\n")
    TokenPrice(response.response_metadata)
```

## 종료 단계

```python
del counter
del bot
del userInput
del data
del counts
del response
```

# 메인 스크립트
---

In [1]:
from IPython.display import Markdown, display
from LLMResponse import LLMResponse

In [2]:
bot = LLMResponse()

items : 83, colours : 24, materials : 23


In [9]:
# 답변 생성
response = bot.GetLLMResponse(
    userInput = "겨울 맑은 날 출근 코디",
    max_scroll_count = 5,
    model_name = "openai:gpt-4.1-nano",
    max_tokens = 1024
)

# 아이템 언급 수 상위 5개 출력
display(response[1])
print("=" * 20)

# 색상 언급 수 상위 5개 출력
display(response[2])
print("=" * 20)

# 재질 언급 수 상위 5개 출력
display(response[3])
print("=" * 20)

# LLM 답변 출력
display(Markdown(response[0]))

니트     550
코트     142
패딩     132
자켓     126
운동화    108
Name: items, dtype: int64

화이트    125
그레이    109
블랙      70
크림      47
브라운     42
Name: colours, dtype: int64

울     460
퍼     187
면     122
다운     48
가죽     47
Name: materials, dtype: int64

```검색 키워드 >> 겨울 맑은 날 출근 코디```

네이버 블로그 내용을 요약하자면, 겨울 출근 패션은 따뜻함과 세련됨을 동시에 갖추는 것이 중요하며, 특히 니트, 코트, 자켓, 패딩, 운동화가 자주 언급됩니다. 색상은 화이트, 그레이, 블랙, 크림, 브라운이 선호되고, 울, 퍼, 면, 다운, 가죽 소재가 주로 사용됩니다. 깔끔하고 실용적이면서도 세련된 스타일을 위해 고급 소재와 무난한 컬러를 조합하는 것이 핵심입니다.

---

## "겨울 맑은 날 출근 코디" 추천 아이템 및 스타일링

### 1. 상의: 화이트 또는 그레이 니트
- 소재: 울 또는 면 혼방
- 특징: 깔끔하고 따뜻한 느낌, 포멀한 오피스룩에 적합

### 2. 아우터: 블랙 또는 크림 컬러의 울 코트 또는 자켓
- 소재: 울 또는 가죽 트리밍이 가미된 울 코트
- 특징: 세련되고 포멀한 분위기 연출

### 3. 하의: 슬림핏 블랙 또는 그레이 팬츠
- 소재: 울 또는 면 혼방
- 특징: 깔끔한 실루엣, 활동성 좋음

### 4. 신발: 운동화 또는 로퍼
- 소재: 가죽 또는 고무
- 특징: 편안함과 세련됨을 동시에

### 5. 액세서리: 심플한 가죽 벨트 또는 스카프
- 색상: 블랙, 브라운, 크림
- 특징: 포인트 주기 좋음

---

## 피해야 될 소재
- 지나치게 캐주얼하거나 반짝이는 퍼 소재
- 너무 두꺼운 패딩 또는 비포멀한 운동화
- 과한 패턴 또는 화려한 프린트

---

## 코디맵 이미지 예시
아래 이미지는 "겨울 출근 맑은 날" 스타일링 참고용입니다.

![겨울 출근 맵](https://images.unsplash.com/photo-1606788075761-4f7b7b7f4f4f?ixlib=rb-4.0.3&auto=format&fit=crop&w=800&q=80)

---

## 추천 상품 링크 (예시)
| 아이템 | 상품명 | 링크 |
|---|---|---|
| 니트 | [울 혼방 화이트 니트](https://search.shopping.naver.com/search/all?query=울+혼방+화이트+니트) | ![이미지](https://via.placeholder.com/100) |
| 코트 | [크림 울 코트](https://search.shopping.naver.com/search/all?query=크림+울+코트) | ![이미지](https://via.placeholder.com/100) |
| 팬츠 | [슬림핏 블랙 울 팬츠](https://search.shopping.naver.com/search/all?query=슬림핏+블랙+울+팬츠) | ![이미지](https://via.placeholder.com/100) |
| 신발 | [가죽 운동화](https://search.shopping.naver.com/search/all?query=가죽+운동화) | ![이미지](https://via.placeholder.com/100) |
| 액세서리 | [심플 가죽 벨트](https://search.shopping.naver.com/search/all?query=심플+가죽+벨트) | ![이미지](https://via.placeholder.com/100) |

---

## 결론
"겨울 맑은 날 출근 코디"는 화이트 또는 그레이 니트와 크림 또는 블랙 울 코트, 슬림핏 팬츠, 편안한 운동화 또는 로퍼로 깔끔하고 따뜻한 오피스룩을 완성하는 것이 핵심입니다. 무난한 컬러와 고급 소재를 활용하여 세련된 이미지를 연출하세요!

```
토큰 사용량
입력 토큰 | 11557 | US$0.0023114000000000003
출력 토큰 | 877 | US$0.0007016
총 토큰 | 12434 | US$0.0030130000000000005
```

In [10]:
print(response[0], "\n")
print(response[1], "\n")
print(response[2], "\n")
print(response[3])

```검색 키워드 >> 겨울 맑은 날 출근 코디```

네이버 블로그 내용을 요약하자면, 겨울 출근 패션은 따뜻함과 세련됨을 동시에 갖추는 것이 중요하며, 특히 니트, 코트, 자켓, 패딩, 운동화가 자주 언급됩니다. 색상은 화이트, 그레이, 블랙, 크림, 브라운이 선호되고, 울, 퍼, 면, 다운, 가죽 소재가 주로 사용됩니다. 깔끔하고 실용적이면서도 세련된 스타일을 위해 고급 소재와 무난한 컬러를 조합하는 것이 핵심입니다.

---

## "겨울 맑은 날 출근 코디" 추천 아이템 및 스타일링

### 1. 상의: 화이트 또는 그레이 니트
- 소재: 울 또는 면 혼방
- 특징: 깔끔하고 따뜻한 느낌, 포멀한 오피스룩에 적합

### 2. 아우터: 블랙 또는 크림 컬러의 울 코트 또는 자켓
- 소재: 울 또는 가죽 트리밍이 가미된 울 코트
- 특징: 세련되고 포멀한 분위기 연출

### 3. 하의: 슬림핏 블랙 또는 그레이 팬츠
- 소재: 울 또는 면 혼방
- 특징: 깔끔한 실루엣, 활동성 좋음

### 4. 신발: 운동화 또는 로퍼
- 소재: 가죽 또는 고무
- 특징: 편안함과 세련됨을 동시에

### 5. 액세서리: 심플한 가죽 벨트 또는 스카프
- 색상: 블랙, 브라운, 크림
- 특징: 포인트 주기 좋음

---

## 피해야 될 소재
- 지나치게 캐주얼하거나 반짝이는 퍼 소재
- 너무 두꺼운 패딩 또는 비포멀한 운동화
- 과한 패턴 또는 화려한 프린트

---

## 코디맵 이미지 예시
아래 이미지는 "겨울 출근 맑은 날" 스타일링 참고용입니다.

![겨울 출근 맵](https://images.unsplash.com/photo-1606788075761-4f7b7b7f4f4f?ixlib=rb-4.0.3&auto=format&fit=crop&w=800&q=80)

---

## 추천 상품 링크 (예시)
| 아이템 | 상품명 | 링크 |
|---|---|---|
| 니트 | [울 혼방 화이트 니트](https://search.shopping.naver.com/s